# Imports 

In [1]:
import pandas as pd
pd.set_option("display.max_colwidth", None)

from tqdm import tqdm
from typing import Optional

from ravenpackapi import Dataset, RPApi
api = RPApi(api_key="MY_API_KEY", product="edge")

# Build an RP dataset

In [2]:
# Crypto coins from the target currency pairs.
coins_to_rp_entity_ids = {
    "APE": "F1E044",
    "AVAX": "E05E4B",
    "AXS": "27EA56",
    "BAKE": "A339AD",
    "BNB": "A2BF5C",
    "BTC": "A25816",
    "BUSD": "FCB6D2",
    "CRV": "0B364F",
    "CTK": "EED9D1",
    "DOGE": "2070E1",
    "DOT": "AA8895",
    # "DYDX" - missing from RP
    "ETH": "0A2CF4",
    "FTM": "ECE804",
    "GMT": "5687B4",
    "LINK": "E7792B",
    "MATIC": "AB69E3",
    "NEAR": "87A7FA",
    "OGN": "32B6C7",
    "RUNE": "BA701C",
    "SAND": "F85EC6",
    "SOL": "BF05E7",
    "STORJ": "49749C",
    # "UNFI" - missing from RP
    "USDT": "9D4B6F",
    "WAVES": "1E4A65",
    "XRP": "547BE3"
}

In [3]:
ds = api.create_dataset(
    Dataset(
        product="edge",
        name="Crypto Dataset",
        fields=[
                "timestamp_utc", "rp_document_id", "rp_entity_id", "entity_type", "entity_name",
                "entity_relevance", "entity_sentiment", "entity_sentiment_confidence", "entity_text_sentiment",
                "entity_text_sent_confidence", "event_relevance", "event_sentiment", "event_risk", 
                "topic", "group", "type", "sub_type", "role", "category", 
                "fact_level", "event_similarity_key", "event_similarity_days", "event_text", 
                "document_sentiment", "document_sentiment_confidence", "composite_sentiment_score", 
                "sentiment_impact_projection", "rp_source_id", "source_name", "rp_provider_id"
        ],
        filters={
            "$and": [
              {
                "entity_relevance": {
                  "$gte": 75
                }
              },
              {
                "event_relevance": {
                  "$gte": 75
                }
              },
              {
                "rp_entity_id": {
                  "$in": list(coins_to_rp_entity_ids.values())
                }
              }
            ]
          },
        frequency="granular"
    )
)
print("Dataset created", ds)

Dataset created Dataset: Crypto Dataset [293BE43BBBE991B689351596682CC618]


In [4]:
%%time
job = ds.request_datafile(
    start_date="2017-01-01",
    end_date="2023-01-01",
)

with open("rp_crypto_data_2017_2023.csv", "a") as fp:
    job.save_to_file(filename=fp.name)

CPU times: user 3.46 s, sys: 1.28 s, total: 4.74 s
Wall time: 2min 34s


# Study event coverage

In [5]:
# Extracted for the target coins, with entity relevance >= 75,
# event relevance >=75, from 2017-01-01 to 2023-01-01.
rp_data = pd.read_csv("rp_crypto_data_2017_2023.csv")
print(rp_data.shape)
rp_data.head()

(456292, 30)


,TIMESTAMP_UTC,RP_DOCUMENT_ID,RP_ENTITY_ID,ENTITY_TYPE,ENTITY_NAME,ENTITY_RELEVANCE,ENTITY_SENTIMENT,ENTITY_SENTIMENT_CONFIDENCE,ENTITY_TEXT_SENTIMENT,ENTITY_TEXT_SENT_CONFIDENCE,...,EVENT_SIMILARITY_KEY,EVENT_SIMILARITY_DAYS,EVENT_TEXT,DOCUMENT_SENTIMENT,DOCUMENT_SENTIMENT_CONFIDENCE,COMPOSITE_SENTIMENT_SCORE,SENTIMENT_IMPACT_PROJECTION,RP_SOURCE_ID,SOURCE_NAME,RP_PROVIDER_ID
0,2017-01-01 14:04:49.954,CC3F32563BC7285D78BCE266789B3209,A25816,CURR,Bitcoin,99,0.21,0.55,0.16,0.43,...,20B63382F9CC75588A51C917BDC6A9FB,0.61389,Bitcoin prices rose more than 100% in 2016,0.12,0.53,0.10,-0.14,C5F8ED,CoinDesk Via MoreOver,MRVR
1,2017-01-01 16:04:59.930,6A4E7370E2C321496D5B0E0DE0042F30,A25816,CURR,Bitcoin,89,0.27,0.59,0.24,0.54,...,2343C5790537BAC29BA23092A3CDF998,365.00000,"Bitcoin would hit $1,000 by December",0.18,0.51,0.10,-0.10,C5F8ED,CoinDesk Via MoreOver,MRVR
2,2017-01-01 16:44:36.473,05F932827990A75FF0561B8E6A4034ED,A25816,CURR,Bitcoin,100,0.16,0.46,-0.04,0.01,...,F0C38EF25F72AE5133CA0075C4B0497F,0.48081,"Bitcoin prices will peak at $3,000 in 2017",0.00,0.11,0.10,-0.08,356E4C,SiliconANGLE,MRVR
3,2017-01-01 17:45:54.219,F3CAAD3F21E10BA0819A856DACC3A475,A25816,CURR,Bitcoin,100,0.02,0.00,-0.02,0.02,...,A7C9FB25B666C1F153C5DF07E3AD5AEE,0.89018,Bitcoin Soars Above $1000 In,0.06,0.16,0.12,-0.48,5E506B,Zero Hedge,MRVR
4,2017-01-01 17:46:29.293,3FB3865528C69BCF9CAF8910B7CDAE0A,A25816,CURR,Bitcoin,100,0.02,0.01,-0.02,0.02,...,A7C9FB25B666C1F153C5DF07E3AD5AEE,0.00041,Bitcoin Soars Above $1000 In,0.05,0.14,0.12,-0.48,B1456F,Peoples Trust Toronto,MRVR


In [6]:
rp_data["CATEGORY"].value_counts()

currency-rate-appreciate                    194117
currency-rate-depreciate                    188887
currency-rate                                34064
currency-guidance                            14940
currency-guidance-appreciate                 13074
currency-adoption-currency                    4488
currency-guidance-depreciate                  3284
currency-rate-volatility                       930
currency-rate-volatility-down                  723
technical-price-level-support-bullish          672
currency-rate-volatility-up                    351
technical-price-level-resistance-bullish       184
technical-price-level-resistance-bearish       145
ico-currency                                   143
technical-price-level-support-bearish           66
technical-view                                  63
relative-strength-index-oversold                23
technical-view-bearish                          22
ico-completed-currency                          22
currency-valuation-devalue     

In [7]:
for category in rp_data["CATEGORY"].value_counts().index:
    print(category)
    rp_data_slice = rp_data[rp_data["CATEGORY"] == category]
    rp_data_slice = rp_data_slice.drop_duplicates("EVENT_SIMILARITY_KEY")
    print(list(rp_data_slice["EVENT_TEXT"].unique())[:10])
    print()

currency-rate-appreciate
['Bitcoin prices rose more than 100% in 2016', 'Bitcoin prices will peak at $3,000 in 2017', 'Bitcoin Soars Above $1000 In', 'Bitcoin have surged since March 2016', 'Bitcoin Price Tops $1,000 in First Day of 2017', 'Bitcoin bounces back despite', 'Bitcoin - a web-based "cryptocurrency" that has no central authority, relying instead on thousands of computers across the world that validate transactions and add new bitcoins to the system - jumped 2.5 percent', 'Bitcoin grows over holidays, hits $1,000 for first time in years', 'Bitcoin breaks above $1,000', 'Bitcoin Breaks $1,000 On the First Day of 2017']

currency-rate-depreciate
['2013, bitcoin plummeted from', 'Bitcoins, and regular currencies sliding against the US dollar', 'Bitcoin fell more than 20 percent in the space of four hours', 'Bitcoin Suffers Biggest Fall in Two Years', 'Bitcoin tanked as much as 23 percent Thursday afternoon after', 'On January 5, 2017 at 13:53 GMT News Bitcoin prices tumbled shar

['Ethusd is arguably breaking upchannel support (on the weekly', 'Btcusd has just broken upchannel support (on the weekly', 'Etc/BTC will have to break a very strong support at 9,685', 'Eth/BTC pair is at 0.026 (price congestion support', 'Xrp/BTC Find Support', 'Bch/BTC Has Found Support Above &#8383', 'For June 2 LTC/BTC has found support above the 200-week', 'Xmr/BTC Attempts To Find Support', 'Ltc/BTC Finds Support Above &#8383', 'Trx/BTC Bounces at the First Significant Support']

technical-view
['Eth/USD rangebound, break above', 'Btc/USD rangebound, upside capped by', 'Btc/USD remains rangebound for the week', 'Xrp/USD Range-Bound, Accumulation Ready for', 'Doge/USD has strict range-bound trading since past 12 days']

relative-strength-index-oversold
['Ltc/BTC is already heavily overbought as shown by the RSI', "Since late April, bitcoin's relative strength index (RSI) has fallen below 60", 'Btcusd pair is now extremely oversold, with the Relative Strength Index', "Xrp/USD is ma

# Case study with crypto-related terms

In [8]:
def show_events_with_term(df: pd.DataFrame, include: str, exclude: Optional[str] = None) -> pd.DataFrame:
    """
    Fetch examples of events with a given term.
    
    :param df: the dataframe with the RP data
    :param include: the term to include in the events
    :param exclude: the term to exclude from the events
    :return: a dataframe slice with the events related to the given term
    """
    # Select events with the term.
    df_term = df[df["EVENT_TEXT"].apply(lambda x: include in x.lower())]
    if exclude is not None:
        # Exclude events with the blacklisted term.
        df_term = df_term[df_term["EVENT_TEXT"].apply(lambda x: exclude not in x.lower())]
    # Remove duplicate events.
    df_term = df_term.drop_duplicates("EVENT_SIMILARITY_KEY")
    df_term = df_term.assign(term=include)
    res = df_term[["term", "EVENT_TEXT", "ENTITY_NAME", "CATEGORY", "EVENT_SENTIMENT"]]
    return res

In [9]:
show_events_with_term(rp_data, include="halving").head(10)

,term,EVENT_TEXT,ENTITY_NAME,CATEGORY,EVENT_SENTIMENT
43166,halving,"Bitcoin prices halving and falling to the $10,000",Bitcoin,currency-rate-depreciate,-0.50
70639,halving,"Ethereum's price action closely, also halving its market cap from just over $8 billion on July 17th",Ethereum,currency-rate-depreciate,-0.50
92343,halving,Bitcoin has traditionally starting pumping around 1 year on average before it's halving,Bitcoin,currency-rate-depreciate,-0.50
92857,halving,Cites 2020 BTC Halving as,Bitcoin,currency-rate-depreciate,-0.50
93157,halving,In February BTC is halving in a little,Bitcoin,currency-rate-depreciate,-0.50
96652,halving,Bitcoin is going to be halving as soon as in May 2020,Bitcoin,currency-rate-depreciate,-0.50
96920,halving,Btc always surges higher ahead of Bitcoin halving,Bitcoin,currency-rate-appreciate,0.46
97582,halving,Btc Price's Historical Block Reward Halving Performance ' (before the May 2020 halving,Bitcoin,currency-rate-depreciate,-0.40
98415,halving,"Bitcoin Will Hit $55,000 After The Bitcoin Halving In 2020",Bitcoin,currency-guidance,0.00
101048,halving,"Bitcoin peaking again shortly after halving, closer to July 2",Bitcoin,currency-rate-appreciate,0.46


In [10]:
show_events_with_term(rp_data, include="reward", exclude="halving").head(10)

,term,EVENT_TEXT,ENTITY_NAME,CATEGORY,EVENT_SENTIMENT
94104,reward,Ethereum Mining Rewards Hit Record Low,Ethereum,currency-rate-depreciate,-0.50
109835,reward,Bitcoin rewards for miners will drop down to 6.25 BTC,Bitcoin,currency-rate-depreciate,-0.50
153713,reward,"Bitcoin, which is about to halve its mining reward on May 12",Bitcoin,currency-rate-depreciate,-0.50
163238,reward,"Three months after Bitcoin halved, slashing miner rewards by",Bitcoin,currency-rate-depreciate,-0.50
166633,reward,Bitcoin Rewards Program Hits New Record,Bitcoin,currency-rate-appreciate,0.46
295311,reward,The Bitcoin protocol automatically halves the reward after every four years,Bitcoin,currency-rate-depreciate,-0.50
395833,reward,Bitcoin halves the rewards after,Bitcoin,currency-rate-depreciate,-0.50
435945,reward,Ethereum staking rewards rise with leveraged stakers seeing 11% APR,Ethereum,currency-rate-appreciate,0.46
447808,reward,"Bitcoins, which includes block rewards and transaction fees, dropped to $11.67 million",Bitcoin,currency-rate-depreciate,-0.50


In [11]:
show_events_with_term(rp_data, include="fork").head(10)

,term,EVENT_TEXT,ENTITY_NAME,CATEGORY,EVENT_SENTIMENT
7368,fork,Bitcoin fork could end up,Bitcoin,currency-guidance-appreciate,0.55
7692,fork,"Bitcoin hardfork, August 2, the price recovered to $2,890",Bitcoin,currency-rate-appreciate,0.67
18389,fork,"Bitcoin cash, which came into being just months ago after bitcoin's soft fork, recently exploded higher by 300% in a matter of days, briefly surpassing the market cap of Ethereum",Ethereum,currency-rate-depreciate,-0.50
44197,fork,Bitcoin could fork up to 50 times in 2018,Bitcoin,currency-guidance-appreciate,0.55
49994,fork,Bitcoin fork dropped to a mere $105,Bitcoin,currency-rate-depreciate,-0.50
91057,fork,"Ethereum, which was expected to maintain a positive upward price movement in anticipation of its Constantinople hard fork on January 14, was overtaken by",Ethereum,currency-rate-appreciate,0.46
91384,fork,Ethereum price - which had traded up in advance of the fork - plunged following,Ethereum,currency-rate-depreciate,-0.50
92934,fork,Bitcoin Forks Surged At Price All-Time High,Bitcoin,currency-rate-appreciate,0.46
122071,fork,Bitcoin fork plunged to as low as $255.10 on July 15,Bitcoin,currency-rate-depreciate,-0.50
283924,fork,Ethereum to plunge dramatically after London hard fork to 'lows under 00,Ethereum,currency-rate-depreciate,-0.57


In [12]:
show_events_with_term(rp_data, include="hodl").head(10)

,term,EVENT_TEXT,ENTITY_NAME,CATEGORY,EVENT_SENTIMENT
137055,hodl,Bitcoin HODLERs remain unmoved despite,Bitcoin,currency-rate,0.00
156656,hodl,Chainlink (LINK) Hodlers Increase by 100% in 2020,ChainLink,currency-rate-appreciate,0.46
202748,hodl,Bitcoin HODLers Rise to a 3.5 Year-High,Bitcoin,currency-rate-appreciate,0.46
401321,hodl,Bitcoin HODLers are holding massive unrealized losses,Bitcoin,currency-rate-depreciate,-0.50
429423,hodl,Bitcoin HODLed hit a five-year record high at,Bitcoin,currency-rate-appreciate,0.46


In [13]:
show_events_with_term(rp_data, include="liquidat", exclude="price").head(10)

,term,EVENT_TEXT,ENTITY_NAME,CATEGORY,EVENT_SENTIMENT
90210,liquidat,Xrp Liquidating Below 40 Cents,Ripple,currency-rate-depreciate,-0.50
104379,liquidat,Btc Bulls under Liquidation Pressure,Bitcoin,currency-guidance-depreciate,-0.60
107676,liquidat,Ethereum (ETH) up 16.8 percent Cryptopia liquidates after Jan 2019,Ethereum,currency-rate-appreciate,0.46
144474,liquidat,"Xbt contracts liquidated According to Datamish, a data aggregator, $104 million USD",Bitcoin,currency-rate-depreciate,-0.50
155803,liquidat,"Bitcoin market began to fall, liquidations on",Bitcoin,currency-rate-depreciate,-0.50
216210,liquidat,"Bitcoin fell to $45K, liquidating $5.9B in futures 2/23/2021",Bitcoin,currency-rate-depreciate,-0.50
238414,liquidat,"Dogecoin liquidations briefly surpassed Bitcoin, indicating high demand for",Dogecoin,currency-rate-appreciate,0.46
268369,liquidat,Bitcoin (BTC) Slips Another 8% With 75% Long Positions Getting Liquidated In Last One Hour,Bitcoin,currency-rate-depreciate,-0.50
298256,liquidat,"Btc Plummets To $45,000 As Liquidations Soar To $312 Million In An Hour",Bitcoin,currency-rate-depreciate,-0.50
298316,liquidat,"Btc Plummets To $45,000 As Liquidations Soar To $312 Million In An Hour",Bitcoin,currency-rate-depreciate,-0.50


In [14]:
show_events_with_term(rp_data, include="miner").head(10)

,term,EVENT_TEXT,ENTITY_NAME,CATEGORY,EVENT_SENTIMENT
3759,miner,"Ethereum miners, again, can increase",Ethereum,currency-guidance-appreciate,0.55
6179,miner,Bitcoin Miner MGT Capital Shoots Up 27%,Bitcoin,currency-rate-appreciate,0.46
73967,miner,Bitcoin Cash miner surpassing BTC.top as,Bitcoin,currency-rate-depreciate,-0.50
78529,miner,Bitcoin miners hit $4.7B,Bitcoin,currency-rate,0.00
78545,miner,Btc miners have already reached $4.7 billion in the first three quarters of 2018,Bitcoin,currency-rate,0.00
84178,miner,Bitcoin miners shut down due to,Bitcoin,currency-rate-depreciate,-0.50
88246,miner,Bitcoin miner revenues hit $4.7 billion through the end of September,Bitcoin,currency-rate-depreciate,-0.50
96119,miner,Bitcoin miner revenues plummeted to a 19-month low in February,Bitcoin,currency-rate-depreciate,-0.50
96452,miner,Bitcoin miner revenues see growth after,Bitcoin,currency-guidance-appreciate,0.55
96517,miner,Bitcoin miners saw a 7% rise in profits in February,Bitcoin,currency-guidance-appreciate,0.55


In [15]:
show_events_with_term(rp_data, include="mining").head(10)

,term,EVENT_TEXT,ENTITY_NAME,CATEGORY,EVENT_SENTIMENT
3870,mining,"Bitcoin bid continuing to rise, Asrock, which offers special panels for mining machines, continued",Bitcoin,currency-rate-appreciate,0.46
6149,mining,Bitcoin Mining Facility Giga Watt Raises $11m in First Weeks of Crowdsale,Bitcoin,ico-currency,0.51
8988,mining,Bitcoin mining boom last quarter,Bitcoin,currency-rate-appreciate,0.46
9012,mining,"Bitcoin mining boom, have surged, hitting a record high of CNY 51.50 on Friday",Bitcoin,currency-rate-appreciate,0.46
11848,mining,North Korea has been engaging in bitcoin mining to earn foreign currency,Bitcoin,currency-adoption-currency,0.46
11881,mining,Ethereum mining difficulty reaches lofty new heights,Ethereum,currency-rate-appreciate,0.46
12878,mining,Canada's Hive Switches From Mining Gold to Bitcoin,Bitcoin,currency-adoption-currency,0.46
24082,mining,"Bitcoin mining machine, with revenues hitting a record high of CNY 3.414bn in the third quarter",Bitcoin,currency-rate-appreciate,0.46
31577,mining,Bitcoin mining may unexpectedly heat up,Bitcoin,currency-guidance-appreciate,0.55
38378,mining,Bitcoin mining venture's revenues hit record high in December,Bitcoin,currency-rate-appreciate,0.46


In [16]:
show_events_with_term(rp_data, include="holder").head(10)

,term,EVENT_TEXT,ENTITY_NAME,CATEGORY,EVENT_SENTIMENT
53205,holder,Bitcoin holders has since shed more than $400 million,Bitcoin,currency-rate,0.00
82300,holder,"Bitcoin, the virtual currency's representative shareholder, fell to its lowest level in more than a year",Bitcoin,currency-rate-depreciate,-0.50
90367,holder,"Bitcoin, which many investors and holders are hoping is going to begin 2019 with a bang, has leaped sharply over the last 24 hours",Bitcoin,currency-rate-appreciate,0.46
91091,holder,Bitcoin (BTC) Long-Term Holders Rise Are Rising,Bitcoin,currency-rate-appreciate,0.46
101431,holder,Holder projects BTC to hit $98 million in 20 years,Bitcoin,currency-guidance,0.00
110485,holder,Xrp holders are likely going to end up having hefty bags by next month,Ripple,currency-rate-appreciate,0.46
111166,holder,Bitcoin Holders Rising after November 2018,Bitcoin,currency-rate-appreciate,0.46
133734,holder,"Bitcoin holders with over 1,000 BTC has surged since early 2019 amidst",Bitcoin,currency-rate-appreciate,0.46
139866,holder,Ethereum retail holders' cumulative balance rose by 6.63% in 2019,Ethereum,currency-rate-appreciate,0.46
158242,holder,Polkadot Holders Votes Could Scale,Polkadot,currency-guidance-appreciate,0.55


In [17]:
show_events_with_term(rp_data, include="whale").head(10)

,term,EVENT_TEXT,ENTITY_NAME,CATEGORY,EVENT_SENTIMENT
144149,whale,Bitcoin halving: Whales may tire of,Bitcoin,currency-rate-depreciate,-0.50
149916,whale,Bitcoin (BTC) whale numbers have hit a two-year high as,Bitcoin,currency-rate-appreciate,0.46
154563,whale,Ethereum Whales cumulative holdings touch 10-months high,Ethereum,currency-rate-appreciate,0.46
173058,whale,Eth Whales Drops to a 3 Month Low,Ethereum,currency-rate-depreciate,-0.50
174721,whale,Ripple (XRP) Whale Addresses Reach New All-Time High,Ripple,currency-rate-appreciate,0.46
181083,whale,Chainlink Whales are Growing,ChainLink,currency-rate-appreciate,0.46
193095,whale,"Bitcoin received by Whale wallets (holding more than 1,000 BTC",Bitcoin,currency-rate-appreciate,0.46
228612,whale,Ethereum Whales Decline as ETH on Exchanges Hits 19 Month Low,Ethereum,currency-rate-depreciate,-0.50
229211,whale,Btc ) has been under severe selling pressure by whales for the past two months,Bitcoin,currency-guidance-depreciate,-0.60
243599,whale,"Dogecoin whales, which we've dubbed ""Bs4,"" is alone holding more than 3.1 billion DOGE",Dogecoin,currency-rate,0.00


In [18]:
show_events_with_term(rp_data, include="transaction").head(10)

,term,EVENT_TEXT,ENTITY_NAME,CATEGORY,EVENT_SENTIMENT
27,transaction,"Bitcoin - a web-based ""cryptocurrency"" that has no central authority, relying instead on thousands of computers across the world that validate transactions and add new bitcoins to the system - jumped 2.5 percent",Bitcoin,currency-rate-appreciate,0.46
60,transaction,"Bitcoin - a web-based ""cryptocurrency"" that has no central authority, relying instead on thousands of computers across the world that validate transactions and add new bitcoins to the system - jumped 2.5 percent",Bitcoin,currency-rate-appreciate,0.46
211,transaction,"Bitcoin - a web-based ""cryptocurrency"" that has no central authority, relying instead on thousands of computers across the world that validate transactions and add new bitcoins to the system - jumped 2.5 percent",Bitcoin,currency-rate-appreciate,0.46
229,transaction,"Bitcoin, a virtual currency used for online transactions, crossed the $1,000 mark on Sunday",Bitcoin,currency-rate-appreciate,0.46
279,transaction,Bitcoin - an Internet-developed cryptomoneda that is not dominated by any central bank and relies on thousands of computer electronic transactions in computers the world - advanced on Sunday night,Bitcoin,currency-rate-appreciate,0.46
808,transaction,"Bitcoin, the virtual currency used for Internet transactions, surpassed $1,100 yesterday",Bitcoin,currency-rate-appreciate,0.46
3896,transaction,"Bitcoin is 1.2% per transaction, lower",Bitcoin,currency-rate-depreciate,-0.50
10225,transaction,"Bitcoin Transactions Below $600 Sep 7, 2017 at 19:00 UTC",Bitcoin,currency-rate,0.00
22121,transaction,Bitcoin transactions have reached $9.75 billion in the past 24 hours,Bitcoin,currency-rate-appreciate,0.46
23698,transaction,"Bitcoin, jumped to $11,800 in transactions Sunday afternoon",Bitcoin,currency-rate-appreciate,0.46


In [19]:
show_events_with_term(rp_data, include="accumul").head(10)

,term,EVENT_TEXT,ENTITY_NAME,CATEGORY,EVENT_SENTIMENT
25250,accumul,"Bitcoin accumulates gains greater than 1,400% this year",Bitcoin,currency-rate-appreciate,0.46
40540,accumul,Bitcoin cools and accumulates 36% drop from the December,Bitcoin,currency-rate-depreciate,-0.50
42104,accumul,Bitcoin has accumulated a loss of,Bitcoin,currency-rate-depreciate,-0.50
55906,accumul,"Bitcoin falls under US$ 7,500 and accumulates 30% in the month",Bitcoin,currency-rate-depreciate,-0.50
73641,accumul,Bitcoin has since been accumulating a 60% drop,Bitcoin,currency-rate,0.00
82313,accumul,Bitcoin fails to recover and accumulates 71% loss,Bitcoin,currency-rate-appreciate,0.46
88288,accumul,"Xrp/USD Range-Bound, Accumulation Ready for",Ripple,technical-view,0.00
102010,accumul,"Btc spiked past $5,000, accumulation had only occurred for around 110 days",Bitcoin,currency-rate-appreciate,0.46
106203,accumul,"Btc bottomed around $200, accumulated around $300 for months",Bitcoin,currency-rate-depreciate,-0.50
144829,accumul,Dogecoin Price Accumulates After It Nosedived Below $0.0023,Dogecoin,currency-rate-depreciate,-0.50
